In [1]:
import os
import re
from CurriculumDB.Modelsn4j import *
from neo4j import GraphDatabase
import openpyxl
import docx

In [2]:
URI = "bolt://localhost:7687"
AUTH = ("curriculum", "mycurriculum")

driver = GraphDatabase.driver(URI, auth=AUTH)
print(driver)
print(driver.verify_connectivity())
#help(driver.verify_connectivity)
factory = CurriculumFactory(driver, 'curriculumdb')

None


In [ ]:
factory.g

In [ ]:
modfile = "C:/Users/dmamartin/OneDrive - University of Dundee/Teaching/CurriculumDB/Module Choices Table.xlsx"
workbook=openpyxl.load_workbook(filename=modfile, data_only=True)

In [ ]:
sheet=workbook.active
for r in range(2,38):
    print(r, sheet.cell(row=r, column=4).value)
    credits=20
    code=sheet.cell(row=r, column=4).value
    name=sheet.cell(row=r, column=5).value
    if code == 'BS41010X':
        credits=40
    shelevel = int(code[2])
    scqflevel=shelevel+6
    semester=int(code[3])
    mod=factory.get_or_create_Element('Module', code=code, shelevel=shelevel,
                                      credits=credits, scqflevel=scqflevel,
                                      semester=semester,
                                      name=name, startyear='25/26')

In [ ]:
moduledir = r"C:\Users\dmamartin\OneDrive - University of Dundee\Teaching\CurriculumDB\Modules"
modules = {}
for stage in os.listdir(moduledir):
    print(f'Looking in {stage}')
    if stage.startswith('Non'):
        continue
    for file in os.listdir(os.path.join(moduledir, stage)):
        print(f'Checking {file}')
        if file.endswith('.docx') and file.startswith('BS'):
            print(f'processing {file}')
            doc=docx.Document(os.path.join(moduledir, stage, file))
            structure ={}
            for t in doc.tables:
                for r in range(len(t.rows)):
                    if len(t.row_cells(r))>1:
                        try:
                            structure[t.row_cells(r)[0].text.split()[0]] =t.row_cells(r)[1].text
                        except:
                            print(file,r)
                            continue
            modules[structure['1.2']]=structure

In [83]:
list(modules[m].keys())

['Applicability',
 'Section',
 'Heading',
 '1.1',
 '1.2',
 '1.3',
 '1.4',
 'Indicative',
 '1.7',
 '1.8',
 '1.9',
 '1.10',
 '1.11',
 '1.12',
 '1.13',
 '1.14',
 '1.15',
 '1.16',
 '1.17',
 '1.18',
 '1.19',
 '2.1',
 '2.2',
 '2.3',
 '2.4',
 '2.5',
 '3.1',
 '3.2',
 '3.3',
 '3.4',
 '3.5',
 '3.6',
 '3.7',
 '3.8',
 '3.9',
 '3.10',
 '3.11',
 '3.12',
 '3.13',
 'Version',
 'Status',
 'Date']

In [119]:
ofh = open('newbimsmoduleilos.txt', 'w')
for m in modules:
    ilos = modules[m]['1.6'].replace('\u2010','').split('\n')
    print(m, len(ilos))
    for i in ilos:
        print(m, i, sep='\t', file=ofh)
ofh.close()

BS31042X 9
BS31050X 10
BS31051X 12
BS32060X 17
BS32061X 10
BS32062X 12
BS42040X 6
BS42042X 21
BS42043X 8
BS42044X 9


In [120]:
ilos={}
with open('newbimsmoduleilos.txt') as fh:
    for line in fh:
        if not line.count('\t'):
            continue
        try:
            (module, ilo) = line.strip().split('\t',1)
            if module not in ilos:
                ilos[module]=[]
            ilos[module].append(ilo)
        except:
            print(line)

BS31042X	

BS31042X	

BS31051X	

BS32060X	

BS32060X	

BS32061X	

BS32062X	

BS42040X	

BS42042X	

BS42042X	

BS42043X	



In [121]:
for m in ilos:
    try:
        mod = factory.get_or_create_Element('Module', code=m)
        for i in ilos[m]:
            mod.map_ilo(factory.get_or_create_Element('ModuleILO', outcome=i), year='25/26')
    except:
        print(f'Module {m} does not exist')


In [ ]:
obj=factory.get_element_by_ID("4:4f947b4e-c5e2-45c5-b439-76aa1c96d891:916")



In [ ]:
elementID = "4:4f947b4e-c5e2-45c5-b439-76aa1c96d891:916"
erecords, _, _ = factory.db.execute_query(
            "MATCH (a ) where elementId(a)=$id "
            "RETURN a", id=elementID,
             database_=factory.dbname,
        )

nodeclass = list(erecords[0].items()[0][1].labels)[0]
params = dict(erecords[0].items()[0][1])
params['elementID']=elementID


In [116]:
moduledir = r"C:\Users\dmamartin\OneDrive - University of Dundee\Teaching\CurriculumDB\New Biomed\Modules\NEW DRAFT MODULES"
modules = {}
for file in os.listdir(moduledir):
#    print(f'Looking in {stage}')
#    if stage.startswith('Non'):
#        continue
#    for file in os.listdir(os.path.join(moduledir, stage)):
        print(f'Checking {file}')
        if file.endswith('.docx') and file.startswith('BS'):
            print(f'processing {file}')
            doc=docx.Document(os.path.join(moduledir,  file))
            structure ={}
            for t in doc.tables:
                for r in range(len(t.rows)):
                    if len(t.row_cells(r))>1:
                        try:
                            structure[t.row_cells(r)[0].text.split()[0]] =t.row_cells(r)[1].text
                        except:
                            print(file,r)
                            continue
            if structure['1.14'] == '':
                structure['1.14'] = getSemester(os.path.join(moduledir,  file))
            modules[structure['1.2']]=structure
            


Checking BS31042X Advanced Statistics for Biomedical and Life Sciences.docx
processing BS31042X Advanced Statistics for Biomedical and Life Sciences.docx
Checking BS31050X Cardiovascular, Respiratory and Renal Physiology.docx
processing BS31050X Cardiovascular, Respiratory and Renal Physiology.docx
Checking BS31051X Neuro, Endo , GI.docx
processing BS31051X Neuro, Endo , GI.docx
Checking BS32060X Pharmacology.docx
processing BS32060X Pharmacology.docx
Checking BS32061X Neuropharmacology.docx
processing BS32061X Neuropharmacology.docx
Checking BS32062X Cancer Biology and Research.docx
processing BS32062X Cancer Biology and Research.docx
Checking BS42040X Cardiovascular Medicine.docx
processing BS42040X Cardiovascular Medicine.docx
Checking BS42042X Advanced Cancer Biology and Therapeutics.docx
processing BS42042X Advanced Cancer Biology and Therapeutics.docx
Checking BS42043X Mechanisms and disorders of memory.docx
processing BS42043X Mechanisms and disorders of memory.docx
Checking BS4

In [111]:
for m in modules:
    print(m,list(modules[m].keys()))

BS31042X ['Applicability', 'Section', 'Heading', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1.10', '1.11', '1.12', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '2.1', '2.2', '2.3', '2.4', '2.5', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '3.10', '3.11', '3.12', '3.13', 'Version', 'Status', 'Date']
BS31050X ['Applicability', 'Section', 'Heading', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1.10', '1.11', '1.12', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '2.1', '2.2', '2.3', '2.4', '2.5', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '3.10', '3.11', '3.12', '3.13', 'Version', 'Status', 'Date']
BS31051X ['Applicability', 'Section', 'Heading', '1.1', '1.2', '1.3', '1.4', '1.5', '1.6', '1.7', '1.8', '1.9', '1.10', '1.11', '1.12', '1.13', '1.14', '1.15', '1.16', '1.17', '1.18', '1.19', '2.1', '2.2', '2.3', '2.4', '2.5', '3.1', '3.2', '3.3', '3.4', '3.5', '3.6', '3.7', '3.8', '3.9', '3.10', 

In [112]:
for m in modules:
    print(m, modules[m]['1.1'],modules[m]['1.14'] , modules[m]['1.3'])
    try: 
        mod = factory.get_or_create_Element('Module', code=m)
        print(f'{m} in database')
    except:
        pass

BS31042X Advanced Statistics for Biomedical and Biological Sciences Semester 1 (once per academic year) Level 3 SHE3 (SCQF 9) 20 SCQF Credits, 10 ECTS 

BS31042X in database
BS31050X Cardiovascular, Respiratory Systems and Renal Systems Semester 1 (once per academic year) Level 3 SHE3 (SCQF 9) 20 SCQF Credits, 10 ECTS
BS31050X in database
BS31051X Neurophysiology, Endocrine and Gastrointestinal Physiology Semester 1 (once per academic year) Level 3 SHE3 (SCQF 9) 20 SCQF Credits, 10 ECTS
BS31051X in database
BS32060X Pharmacology Semester 2 Block D SHE3 (SCQF level 9); 20 SCQF Credits = 7.5 ECTS Credits
BS32060X in database
BS32061X Neuropharmacology Semester 2 Block D SHE3 (SCQF level 9); 15 SCQF Credits = 7.5 ECTS Credits
BS32061X in database
BS32062X Cancer Biology Semester 2 SCQF level 10; 15 SCQF credits
BS32062X in database
BS42040X Cardiovascular Medicine Semester 2 (once per academic year) Undergraduate. SHE level M, SCQF level 10
No of credits 20
BS42040X in database
BS42042X A

In [114]:
modules[m]['1.6']

'On completion of this module students should exhibit the following:\nKnowledge & understanding\nBe able to critically evaluate central concepts relevant to neuropsychiatric disorders\nHave an advanced appreciation of the mechanisms of action of pharmacotherapies for the treatment of neuropsychiatric disorders. \nBe able to provide an in depth description of the mechanisms of action of anaesthetic and analgesic drugs.\nBe able to demonstrate an advanced understanding of different pain states and the pain pathways.\nSKills\nThe student should be able to access and interpret the scientific literature relevant to neuropsychiatric disorders, anaesthesia and analgesia. Novel information should be interpreted in the context of the knowledge gained from this module as evidenced by essays on selected topics. \nExhibit self-reliance and effective independent study skills. For example, students should be familiar with accessing additional relevant peer reviewed information to enhance their level

In [117]:
modobj ={}

scqf = re.compile(r'SCQF\D+(\d+)\D+(\d+)\D')
delivery = re.compile(r'Semester (\d)')
#for m in modules:
#    print(m)
#for m in ['BS32031']:
for m in modules:
    print(f'processing {m}')
    try:
        
        newmod=modules[m]
    
        match=scqf.search(newmod['1.3'])
        scqflevel=0
        shelevel=0
        credits=0
        if match:
            scqflevel = int(match.group(1))
            shelevel = scqflevel - 6
            credits = int(match.group(2))
        else: 
            shelevel = int(m[2])
            scqflevel = shelevel +6
            print(m,"1.3", newmod['1.3'])
        match=delivery.search(newmod['1.14'])
        semester=0
        if match:
            semester = int(match.group(1))
        else:
            print(m,"1.14",newmod['1.14'])
            semester = int(m[3])
            if not semester:
                semester='both'
        if credits and scqflevel and shelevel and semester is not None:
            newmodparams ={'code':newmod['1.2'], 'name':newmod['1.1'], 'semester':semester, 
                           'credits': credits, 'scqflevel':scqflevel, 'shelevel':shelevel}
            try:
                modobj[m]=factory.get_or_create_Element('Module', code = m)
                modobj[m].update(**newmodparams)
            except:
                modobj[m]=factory.get_or_create_Element('Module', **newmodparams)
    except Exception as e:
        print(newmodparams,e)
        

processing BS31042X
processing BS31050X
processing BS31051X
processing BS32060X
processing BS32061X
processing BS32062X
processing BS42040X
processing BS42042X
processing BS42043X
processing BS42044X


In [65]:
doc=docx.Document(os.path.join(moduledir,  'BS32042X Bioinformatics Methods 2526 DRAFT.docx'))
structure ={}
rawstruct={}
for t in doc.tables:
    for r in range(len(t.rows)):
        if len(t.row_cells(r))>1:
            try:
                structure[t.row_cells(r)[0].text.split()[0]] =t.row_cells(r)[1].text
                rawstruct[t.row_cells(r)[0].text.split()[0]]=t.row_cells(r)[1]
            except:
                print(file,r)
                continue
        

In [31]:
for n in rawstruct['1.14'].iter_inner_content():
    print(n.text)

In [39]:
import zipfile
from bs4 import BeautifulSoup
from xml.dom.minidom import parseString
document = zipfile.ZipFile(os.path.join(moduledir,  'BS31041X - Cell Biology -Module specification.docx'))
xml_data = document.read('word/document.xml')
document.close()

soup = BeautifulSoup(xml_data, 'xml')
gegevens = soup.findAll('dropDownList')     #search dropdownlists (n = 12)

dropdownlist = []
dropdownlistdata = []

for i in gegevens:
    dropdownlist.append(i.find('result'))

dom = parseString(xml_data)


In [85]:
document=None


In [42]:
lines=dom.toprettyxml().split('\n')
count=0
for line in lines:
    if 'Semester' in line:
        print(count, line)
    count+=1
    

3618 									<w:listItem w:displayText="Semester 1 (once per academic year)" w:value="Semester 1 (once per academic year)"/>
3619 									<w:listItem w:displayText="Semester 2 (once per academic year)" w:value="Semester 2 (once per academic year)"/>
3620 									<w:listItem w:displayText="Semester 3 (once per academic year)" w:value="Semester 3 (once per academic year)"/>
3624 									<w:listItem w:displayText="Semester 1 and 2 (twice per academic year)" w:value="Semester 1 and 2 (twice per academic year)"/>
3625 									<w:listItem w:displayText="Semester 1 and 3 (twice per academic year)" w:value="Semester 1 and 3 (twice per academic year)"/>
3626 									<w:listItem w:displayText="Semester 2 and 3 (twice per academic year)" w:value="Semester 2 and 3 (twice per academic year)"/>
3627 									<w:listItem w:displayText="Semester 1,2 and 3 (three times per academic year)" w:value="Semester 1,2 and 3 (three times per academic year)"/>
3639 									<w:t>Semester 1 (once per academ

In [46]:
for line in lines[3550:3650]:
    print(line)

				<w:tc>
					<w:tcPr>
						<w:tcW w:w="1249" w:type="pct"/>
						<w:tcBorders>
							<w:top w:val="single" w:sz="4" w:space="0" w:color="006666"/>
							<w:left w:val="single" w:sz="4" w:space="0" w:color="006666"/>
							<w:bottom w:val="single" w:sz="4" w:space="0" w:color="006666"/>
							<w:right w:val="single" w:sz="4" w:space="0" w:color="006666"/>
						</w:tcBorders>
						<w:hideMark/>
					</w:tcPr>
					<w:p w14:paraId="0AF3CC41" w14:textId="77777777" w:rsidR="006E4680" w:rsidRPr="006E31F1" w:rsidRDefault="006E4680" w:rsidP="006E4680">
						<w:pPr>
							<w:rPr>
								<w:rFonts w:asciiTheme="minorHAnsi" w:hAnsiTheme="minorHAnsi"/>
								<w:sz w:val="20"/>
								<w:szCs w:val="20"/>
								<w:lang w:eastAsia="en-US"/>
							</w:rPr>
						</w:pPr>
						<w:r w:rsidRPr="006E31F1">
							<w:rPr>
								<w:rFonts w:asciiTheme="minorHAnsi" w:hAnsiTheme="minorHAnsi"/>
								<w:sz w:val="20"/>
								<w:szCs w:val="20"/>
								<w:lang w:eastAsia="en-US"/>


In [49]:
trlist = dom.getElementsByTagName('w:tr')

In [62]:
def getSemester(file):
    document = zipfile.ZipFile(file)
    xml_data = document.read('word/document.xml')
    document.close()
    dom = parseString(xml_data)
    trlist = dom.getElementsByTagName('w:tr')
    for row in trlist:
        cells = row.getElementsByTagName('w:tc')
        if len(cells )!=2:
            continue
        desc = cells[0].getElementsByTagName('w:t')
        content =cells[1].getElementsByTagName('w:t')
        if len(desc) and len(content):
            section = desc[0].firstChild.data.split()[0]
            if section == '1.14':
                return content[0].firstChild.data
            

In [50]:
trlist

[<DOM Element: w:tr at 0x2746a39f750>,
 <DOM Element: w:tr at 0x2746a3c36b0>,
 <DOM Element: w:tr at 0x2746a3d1310>,
 <DOM Element: w:tr at 0x2746a3e87d0>,
 <DOM Element: w:tr at 0x2746a3eb6b0>,
 <DOM Element: w:tr at 0x2746a3f7430>,
 <DOM Element: w:tr at 0x2746a40e350>,
 <DOM Element: w:tr at 0x2746a4320d0>,
 <DOM Element: w:tr at 0x2746a482ad0>,
 <DOM Element: w:tr at 0x2746a50ea30>,
 <DOM Element: w:tr at 0x2746a542670>,
 <DOM Element: w:tr at 0x2746a54d630>,
 <DOM Element: w:tr at 0x2746a54f6b0>,
 <DOM Element: w:tr at 0x2746a5639d0>,
 <DOM Element: w:tr at 0x2746a57e5d0>,
 <DOM Element: w:tr at 0x2746a5899f0>,
 <DOM Element: w:tr at 0x2746a5b8550>,
 <DOM Element: w:tr at 0x2746a5bb6b0>,
 <DOM Element: w:tr at 0x2746a5d18b0>,
 <DOM Element: w:tr at 0x2746a5d3610>,
 <DOM Element: w:tr at 0x2746a5e0730>,
 <DOM Element: w:tr at 0x2746a5e2530>,
 <DOM Element: w:tr at 0x2746a5f0370>,
 <DOM Element: w:tr at 0x2746a5f2170>,
 <DOM Element: w:tr at 0x2746a5f3d90>,
 <DOM Element: w:tr at 0x

In [37]:
str(xml_data)[95500:98500]

' w:asciiTheme="minorHAnsi" w:hAnsiTheme="minorHAnsi"/><w:sz w:val="20"/><w:szCs w:val="20"/><w:lang w:eastAsia="en-US"/></w:rPr><w:t xml:space="preserve">1.14 When delivered </w:t></w:r></w:p></w:tc><w:tc><w:tcPr><w:tcW w:w="3751" w:type="pct"/><w:tcBorders><w:top w:val="single" w:sz="4" w:space="0" w:color="006666"/><w:left w:val="single" w:sz="4" w:space="0" w:color="006666"/><w:bottom w:val="single" w:sz="4" w:space="0" w:color="006666"/><w:right w:val="single" w:sz="4" w:space="0" w:color="006666"/></w:tcBorders><w:hideMark/></w:tcPr><w:p w14:paraId="0AF3CC42" w14:textId="316F2316" w:rsidR="006E4680" w:rsidRPr="006E31F1" w:rsidRDefault="000F4711" w:rsidP="006E4680"><w:pPr><w:spacing w:after="60"/><w:rPr><w:rFonts w:asciiTheme="minorHAnsi" w:hAnsiTheme="minorHAnsi"/><w:sz w:val="20"/><w:szCs w:val="20"/><w:lang w:eastAsia="en-US"/></w:rPr></w:pPr><w:sdt><w:sdtPr><w:rPr><w:rFonts w:asciiTheme="minorHAnsi" w:hAnsiTheme="minorHAnsi"/><w:sz w:val="20"/><w:szCs w:val="20"/><w:lang w:eas

In [35]:
str(xml_data).find('1.14')

95654